In [2]:
import os
import pandas as pd
from os.path import join
from tta_wrapper_venheads import tta_segmentation
import cv2
from segmentation_models import Unet
from keras import optimizers
import numpy as np
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.advanced_activations import ELU
from keras.models import Model
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, BatchNormalization, Activation, UpSampling2D
from keras.optimizers import SGD
import keras.backend as K
from keras.layers import Input,Dropout,BatchNormalization,Activation,Add
from skimage.io import imread
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
import matplotlib.pyplot as plt
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from tqdm import tqdm
from keras import optimizers
from losses_original import (
    binary_crossentropy,
    dice_loss,
    bce_dice_loss,
    dice_coef,
    weighted_bce_dice_loss,my_iou_metric,
    my_iou_metric_2,IOU_Metric_new,my_iou_metric_bowl,
    competition_metric,
    iou_bce_loss,lovasz_loss
)

number_folds = 5
os.environ["CUDA_VISIBLE_DEVICES"]="1"

Using TensorFlow backend.


In [3]:
df = pd.read_csv('/mnt/ssd1/dataset/salt/sample_submission.csv')
weg_train_img = '/mnt/ssd1/dataset/salt/images-test/'

In [4]:
def custom_pad(img, pad_size=256, mode=3):
    x1 = (pad_size - img.shape[0]) // 2
    x2 = pad_size - img.shape[0] - x1

    if mode == 0:
        img = cv2.copyMakeBorder(img, x1, x2, x1, x2, cv2.BORDER_CONSTANT)

    if mode == 1:
        img = cv2.copyMakeBorder(img, x1, x2, x1, x2, cv2.BORDER_REPLICATE)

    if mode == 2:
        img = cv2.copyMakeBorder(img, x1, x2, x1, x2, cv2.BORDER_REFLECT_101)

    if mode == 3:
        img = cv2.copyMakeBorder(img, x1, x2, 0, 0, cv2.BORDER_REPLICATE)
        img = cv2.copyMakeBorder(img, 0, 0, x1, x2, cv2.BORDER_REFLECT_101)

    if mode == 4:
        img = cv2.copyMakeBorder(img, x1, x2, 0, 0, cv2.BORDER_REFLECT_101)
        img = cv2.copyMakeBorder(img, 0, 0, x1, x2, cv2.BORDER_REPLICATE)

    return img

In [5]:
pad = 77
load_img = lambda im: np.pad(cv2.imread(join(weg_train_img, '{}.png'.format(im))) #[:,:,0:1]\
, ((pad,pad+1),(pad,pad+1),(0,0)), 'constant')
load_mask = lambda im: np.pad(cv2.imread(join(weg_train_mask, '{}.png'.format(im)))[:,:,0]\
                              , ((pad,pad+1),(pad,pad+1)), 'constant')

load_img = lambda im: custom_pad(cv2.imread(join(weg_train_img, '{}.png'.format(im))))
load_mask = lambda im: custom_pad(cv2.imread(join(weg_train_mask, '{}.png'.format(im)))[:,:,0])\

test_ids = df.id.values


In [6]:
weg = 'predictions'


In [8]:
for part in range(2,5):
    for fold in range(4):
        print ('FOLD ' + str(fold))
        model = Unet(backbone_name='resnet34', encoder_weights='imagenet',input_shape=(256,256,3),
                            decoder_use_batchnorm = True,decoder_block_type = 'upsampling')
        model.compile(optimizers.Adam(lr = 0.0001), bce_dice_loss, ['accuracy',competition_metric])
        model.load_weights('{}_origina_model_weights_fold_{}.hdf5'.format(part,fold))
        #tta_model = tta_segmentation(model, h_flip=True, 
         #                    h_shifts=(-10, 10),v_shifts=(-10,10), merge='mean')

        if not os.path.exists(weg+'{}_model_weights_fold_{}'.format(part,fold)): 
            os.mkdir(weg+'{}_model_weights_fold_{}'.format(part,fold))
        for test_id in tqdm(test_ids,miniters=1000):
            test_img =  load_img(test_id)/255.
            test_img = np.expand_dims(test_img,0)
            pred = model.predict(test_img)[0,77:-78,77:-78]
            np.savez_compressed(weg+'{}_model_weights_fold_{}/'.format(part,fold) + test_id,pred)
        print ('Saved for fold '+ str(fold))
        del model

FOLD 0


100%|██████████| 18000/18000 [05:55<00:00, 50.58it/s]


Saved for fold 0
FOLD 1


100%|██████████| 18000/18000 [06:18<00:00, 47.58it/s]


Saved for fold 1
FOLD 2


...

100%|██████████| 18000/18000 [06:42<00:00, 44.67it/s]


Saved for fold 2
FOLD 3


 54%|█████▎    | 9661/18000 [03:39<03:09, 43.98it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100%|██████████| 18000/18000 [07:02<00:00, 42.61it/s]


Saved for fold 1
FOLD 2


 11%|█         | 1946/18000 [00:47<06:35, 40.63it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 47%|████▋     | 8411/18000 [03:23<03:51, 41.33it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [9]:
def RLenc(img, order='F', format=True):
    """
    img is binary mask image, shape (r,c)
    order is down-then-right, i.e. Fortran
    format determines if the order needs to be preformatted (according to submission rules) or not

    returns run length as an array or string (if format is True)
    """
    bytes = img.reshape(img.shape[0] * img.shape[1], order=order)
    runs = []  ## list of run lengths
    r = 0  ## the current run length
    pos = 1  ## count starts from 1 per WK
    for c in bytes:
        if (c == 0):
            if r != 0:
                runs.append((pos, r))
                pos += r
                r = 0
            pos += 1
        else:
            r += 1

    # if last run is unsaved (i.e. data ends with 1)
    if r != 0:
        runs.append((pos, r))
        pos += r
        r = 0

    if format:
        z = ''

        for rr in runs:
            z += '{} {} '.format(rr[0], rr[1])
        return z[:-1]
    else:
        return runs

### Making submit

In [10]:
def rle_encode(im):
    pixels = im.flatten(order = 'F')
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def filter_image(img):
    if img.sum() < 100:
        return np.zeros(img.shape)
    else:
        return img

In [11]:
def rle_encode(im):
    '''
    im: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = im.flatten(order = 'F')
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [17]:
threshold = 0.47
for x in [0]:
    ids_test = test_ids



    def _mask_to_rle_string(mask):
        """Convert boolean/`binary uint` mask to RLE string."""
        # Mask to RLE
        pixels = mask.flatten()
        pixels[0] = 0
        pixels[-1] = 0
        runs = np.where(pixels[1:] != pixels[:-1])[0] + 2
        runs[1::2] = runs[1::2] - runs[:-1:2]

        # RLE to string
        return ' '.join(str(x) for x in runs)

    names = []
    for id in ids_test:
        names.append('{}'.format(id))


    rles = []

    for id_1 in tqdm(ids_test,miniters=1000):
        placeholder = np.ones((101,101,1))
        for fold in range(4):
            placeholder_2 = np.ones((101,101,1))
            for part in range(2,5):
                loaded = np.load(weg+'{}_model_weights_fold_{}/'.format(part,fold) + str(id_1)+'.npz')['arr_0']
                placeholder_2*=loaded
            placeholder_2 = np.power(placeholder_2,1./3)
            prob = placeholder_2#[13:-14,13:-14,:]
            placeholder*=prob
        placeholder = np.power(placeholder,1./4)    
        mask = placeholder > threshold
        #rle = _mask_to_rle_string(mask)
        #rle = rle_encode(filter_image(mask))
        rle = rle_encode(mask)
        if len(rle.split(' ')) < 5:
            rle = ''
        rles.append(rle)

    print("Generating submission file...")
    df = pd.DataFrame({'id': names, 'rle_mask': rles})
    df.to_csv('submit/fold_'+str(fold)+'densenet169.csv.gz', index=False, compression='gzip')
    print('finished_fold_' + str(fold))

100%|██████████| 18000/18000 [04:50<00:00, 62.05it/s]


Generating submission file...
finished_fold_3


In [ ]:
ids_test = test_ids
for id_1 in tqdm(ids_test,miniters=1000):
    placeholder = np.ones((101,101,1))
    for fold in range(4):
        placeholder_2 = np.ones((101,101,1))
        for part in range(2,5):
            loaded = np.load(weg+'{}_model_weights_fold_{}/'.format(part,fold) + str(id_1)+'.npz')['arr_0']
            placeholder_2*=loaded
        placeholder_2 = np.power(placeholder_2,1./3)
        prob = placeholder_2#[13:-14,13:-14,:]
        placeholder*=prob
    placeholder = np.power(placeholder,1./5)    
    np.savez_compressed('total_geom_avg/{}'.format(id_1),placeholder)
    cv2.imwrite("total_geom_avg/{}.png".format(id_1), placeholder)


 10%|▉         | 1771/18000 [00:35<05:22, 50.33it/s]